# Extract text from PDF

### install requirements

In [2]:
!pip install torch transformers datasets pdf2image pymupdf python-Levenshtein nltk

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.5 MB 5.6 MB/s eta 0:00:03
   ------ --------------------------------- 2.9/16.5 MB 8.4 MB/s eta 0:00:02
   ---------- ----------------------------- 4.5/16.5 MB 8.7 MB/s eta 0:00:02
   --------------- ------------------------ 6.3/16.5 MB 8.6 MB/s eta 0:00:02
   ------------------ --------------------- 7.6/16.5 MB 8.2 MB/s eta 0:00:02
   -------------------- ------------------- 8.7/16.5 MB 7.6 MB/s eta 0:00:02
   ------------------------ --------------- 10.0/16.5 MB 7.3 MB/s eta 0:00:01
   --------------------------- ------------ 11.5/16.5 MB 7.4 MB/s eta 0:00:01
   -------------------------------- ------- 13.6/16.5 MB 7.7 MB/s eta 0:00:01
   ------------------------------------ --- 14.9/16.5 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------  16.5/16.5 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 16.5/16.5 MB 7.4 MB/s eta 0:00:00


### Load the model

In [3]:
from transformers import AutoProcessor, VisionEncoderDecoderModel
import torch

In [4]:
processor = AutoProcessor.from_pretrained("facebook/nougat-small")

preprocessor_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

c:\Users\abdel\.conda\envs\ML\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abdel\.cache\huggingface\hub\models--facebook--nougat-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` w

tokenizer_config.json:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [5]:
model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-small")

config.json:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    896,
    672
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "qkv_bias": true,
  "transformers_version": "4.48.1",
  "use_absolute_embeddings": false,
  "window_size": 7
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add_final_layer_norm": true

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0): DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )

### Load PDF

In [9]:
from huggingface_hub import hf_hub_download

filepath = hf_hub_download(repo_id="ysharma/nougat", filename="input/nougat.pdf", repo_type="space")

nougat.pdf:   0%|          | 0.00/4.13M [00:00<?, ?B/s]

C:\Users\abdel\.cache\huggingface\hub\spaces--ysharma--nougat\snapshots\d06487a88e149a541a2803ea037f7a0bbd3b6c2c\input\nougat.pdf


c:\Users\abdel\.conda\envs\ML\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abdel\.cache\huggingface\hub\spaces--ysharma--nougat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [42]:
from typing import Optional, List
import io
import fitz
from pathlib import Path

def rasterize_paper(
    pdf: Path,
    outpath: Optional[Path] = None,
    dpi: int = 96,
    return_pil=False,
    pages=None,
) -> Optional[List[io.BytesIO]]:
    """
    Rasterize a PDF file to PNG images.

    Args:
        pdf (Path): The path to the PDF file.
        outpath (Optional[Path], optional): The output directory. If None, the PIL images will be returned instead. Defaults to None.
        dpi (int, optional): The output DPI. Defaults to 96.
        return_pil (bool, optional): Whether to return the PIL images instead of writing them to disk. Defaults to False.
        pages (Optional[List[int]], optional): The pages to rasterize. If None, all pages will be rasterized. Defaults to None.

    Returns:
        Optional[List[io.BytesIO]]: The PIL images if `return_pil` is True, otherwise None.
    """

    pillow_images = []
    if outpath is None:
        return_pil = True
    try:
        if isinstance(pdf, (str, Path)):
            pdf = fitz.open(pdf)
        if pages is None:
            pages = range(len(pdf))
        for i in pages:
            page_bytes: bytes = pdf[i].get_pixmap(dpi=dpi).pil_tobytes(format="PNG")
            if return_pil:
                pillow_images.append(io.BytesIO(page_bytes))
            else:
                with (outpath / ("%02d.png" % (i + 1))).open("wb") as f:
                    f.write(page_bytes)
    except Exception:
        pass
    if return_pil:
        return pillow_images

In [71]:
from PIL import Image

images = rasterize_paper(pdf="docForTest.pdf", return_pil=True)
images = [Image.open(image) for image in images]


In [72]:
print(images)

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=816x1056 at 0x272C5B8E1F0>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=816x1056 at 0x272C5B8E8E0>]


### Generate the text

In [58]:
pixel_values = processor(images=images, return_tensors="pt").pixel_values
print(pixel_values.shape)

torch.Size([2, 3, 896, 672])


In [18]:
from transformers import StoppingCriteria, StoppingCriteriaList
from collections import defaultdict

class RunningVarTorch:
    def __init__(self, L=15, norm=False):
        self.values = None
        self.L = L
        self.norm = norm

    def push(self, x: torch.Tensor):
        assert x.dim() == 1
        if self.values is None:
            self.values = x[:, None]
        elif self.values.shape[1] < self.L:
            self.values = torch.cat((self.values, x[:, None]), 1)
        else:
            self.values = torch.cat((self.values[:, 1:], x[:, None]), 1)

    def variance(self):
        if self.values is None:
            return
        if self.norm:
            return torch.var(self.values, 1) / self.values.shape[1]
        else:
            return torch.var(self.values, 1)


class StoppingCriteriaScores(StoppingCriteria):
    def __init__(self, threshold: float = 0.015, window_size: int = 200):
        super().__init__()
        self.threshold = threshold
        self.vars = RunningVarTorch(norm=True)
        self.varvars = RunningVarTorch(L=window_size)
        self.stop_inds = defaultdict(int)
        self.stopped = defaultdict(bool)
        self.size = 0
        self.window_size = window_size

    @torch.no_grad()
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        last_scores = scores[-1]
        self.vars.push(last_scores.max(1)[0].float().cpu())
        self.varvars.push(self.vars.variance())
        self.size += 1
        if self.size < self.window_size:
            return False

        varvar = self.varvars.variance()
        for b in range(len(last_scores)):
            if varvar[b] < self.threshold:
                if self.stop_inds[b] > 0 and not self.stopped[b]:
                    self.stopped[b] = self.stop_inds[b] >= self.size
                else:
                    self.stop_inds[b] = int(
                        min(max(self.size, 1) * 1.15 + 150 + self.window_size, 4095)
                    )
            else:
                self.stop_inds[b] = 0
                self.stopped[b] = False
        return all(self.stopped.values()) and len(self.stopped) > 0

In [59]:

# autoregressively generate tokens, with custom stopping criteria (as defined by the Nougat authors)
outputs = model.generate(pixel_values.to(device),
                          min_length=1,
                          max_length=6000,
                          bad_words_ids=[[processor.tokenizer.unk_token_id]],
                          return_dict_in_generate=True,
                          output_scores=True,
                          stopping_criteria=StoppingCriteriaList([StoppingCriteriaScores()]),
)
     

C:\Users\abdel\AppData\Local\Temp\ipykernel_22048\4023200142.py:23: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\ReduceOps.cpp:1823.)
  return torch.var(self.values, 1) / self.values.shape[1]


### post processing

In [68]:
generated = processor.batch_decode(outputs[0], skip_special_tokens=True)
generated = processor.post_process_generation(generated, fix_markdown=False)

In [81]:
file_path = Path("docForTest.pdf")

with open(str(file_path).replace(".pdf",".txt"), "w", encoding="utf-8") as file:
    file.write("".join(generated))

'Postdoctoral researcher - Optimization and automation of agent-based model development for Digital Twins of Mobility\n\nWe are looking for a highly motivated Postdoctoral researcher to join the Center for Sustainable Future Mobility at ETH and work on the Innovation Project funded by Innosuisse. The researcher will focus on applying optimization and machine learning methodology to calibrate and automate agent-based transport model development and collaborate with a unique group of practitioners in developing state-of-the-art regional transport models.\n\n## Project Background\n\nThe Innovation project with three implementation partners (transports publics genevois, transports publics de la region lausannoise, Ville de Lausanne) seeks to develop an open-source collaborative platform for mobility analysis and forecasting. This platform will introduce several innovations: it will use state-of-the-art mobility data, automate data import, enable mobility data analysis, automate calibration